In [27]:
from langchain_openai.llms import OpenAI
import os
import dotenv

dotenv.load_dotenv()

llm = OpenAI(model="gpt-3.5-turbo-0125",api_key=os.getenv("openaiKey"))

In [28]:
from langchain.prompts import PromptTemplate


prompt_template = PromptTemplate.from_template("""
responda a seguinte pergunta do usuário:
    {pergunta}""")

In [29]:
print(prompt_template.format(pergunta="o que um saas?"))


responda a seguinte pergunta do usuário:
    o que um saas?


In [30]:
from langchain.prompts import PromptTemplate


prompt_template = PromptTemplate.from_template("""
responda a seguinte pergunta do usuário em ate {n_palavras} palavras:
    {pergunta}""")

In [31]:
print(prompt_template.format(pergunta="o que um saas?",n_palavras=15))


responda a seguinte pergunta do usuário em ate 15 palavras:
    o que um saas?


In [32]:
prompt_template = PromptTemplate.from_template("""
responda a seguinte pergunta do usuário em ate {n_palavras} palavras:
    {pergunta}""",partial_variables={"n_palavras":10})

In [33]:
print(prompt_template.format(pergunta="o que e langchain?"))


responda a seguinte pergunta do usuário em ate 10 palavras:
    o que e langchain?


### utilizando multiplos prompts

In [34]:
from langchain.prompts import PromptTemplate

template_word_counts = PromptTemplate.from_template("""
responda a pergunta em ate {n_palavras} palavras.
"""
)

template_line_count = PromptTemplate.from_template("""
responda a pergunta em ate {n_linhas} linhas.
""")

template_idiomas = PromptTemplate.from_template("""
retorne a resposta em {idioma}.
""")

template_final = (template_word_counts + template_line_count + template_idiomas + 
                  "responda a pergunta seguindo as instruções {pergunta}")

In [35]:
print(template_final)

input_variables=['idioma', 'n_linhas', 'n_palavras', 'pergunta'] template='\nresponda a pergunta em ate {n_palavras} palavras.\n\nresponda a pergunta em ate {n_linhas} linhas.\n\nretorne a resposta em {idioma}.\nresponda a pergunta seguindo as instruções {pergunta}'


In [37]:
prompt_final = template_final.format(n_palavras=15, n_linhas=1, idioma="inglês", pergunta="O que é o Sol?")
llm.invoke(prompt_final)

NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

### Templates para chat

In [39]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template("essa e minha duvida :{duvida}")
chat_template.format_messages(duvida="quem e voce?")

[HumanMessage(content='essa e minha duvida :quem e voce?')]

In [ ]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system","voce e um assistente ironico e se chama {nome_assistente}"),
    ("human","ola,como vai?"),
    ("ai","estou bem,como posso lhe ajudar?"),
    ("human","{pergunta}")
])

In [41]:
chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], template='voce e um assistente ironico e se chama {nome_assistente}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='ola.como vai?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='estou bem,como posso lhe ajudar?')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], template='{pergunta}'))])

In [42]:
chat_template.format_messages(nome_assistente="baymax",pergunta="qual seu nome?")

[SystemMessage(content='voce e um assistente ironico e se chama baymax'),
 HumanMessage(content='ola.como vai?'),
 AIMessage(content='estou bem,como posso lhe ajudar?'),
 HumanMessage(content='qual seu nome?')]

In [43]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(api_key=os.getenv("openaiKey"))

chat.invoke(chat_template.format_messages(nome_assistente="baymax",pergunta="qual seu nome?"))

AIMessage(content='meu nome e baymax, estou aqui para ajudar. Como posso ser util hoje?', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 54, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e9550a3a-2037-4e60-a5dd-4ee9b4d8e800-0')

In [45]:
exemplos = [
    {"pergunta": "Qual é a maior montanha do mundo, o Monte Everest ou o K2?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 
""", 
    }, 
    {"pergunta": "Quem nasceu primeiro, Charles Darwin ou Albert Einstein?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 
""", 
    }, 
    {"pergunta": "Quem foi o pai de Napoleão Bonaparte?",
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte? 
Resposta intermediária: Napoleão Bonaparte foi um líder militar e imperador francês. 
Pergunta de acompanhamento: Quem foi o pai de Napoleão Bonaparte? 
Resposta intermediária: O pai de Napoleão Bonaparte foi Carlo Buonaparte. 
Então a resposta final é: Carlo Buonaparte 
""", 
    },
    {"pergunta": "Os filmes 'O Senhor dos Anéis' e 'O Hobbit' foram dirigidos pelo mesmo diretor?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem dirigiu 'O Senhor dos Anéis'? 
Resposta intermediária: 'O Senhor dos Anéis' foi dirigido por Peter Jackson. 
Pergunta de acompanhamento: Quem dirigiu 'O Hobbit'? 
Resposta intermediária: 'O Hobbit' também foi dirigido por Peter Jackson. 
Então a resposta final é: Sim 
""",
    },
]

In [46]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(input_variables=["pergunta","resposta"],template="pergunta: {pergunta}\n{resposta}")

In [47]:
prompt = FewShotPromptTemplate(examples=exemplos,
                               example_prompt=example_prompt,
                               suffix="pergunta:{input}",
                               input_variables=["input"])

In [ ]:
prompt_final = PromptTemplate(input_variables=["input"],template=prompt)

In [ ]:
print(prompt_final.format(input="quem e melhor, messi ou cr7?"))

pergunta: Qual é a maior montanha do mundo, o Monte Everest ou o K2?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 


pergunta: Quem nasceu primeiro, Charles Darwin ou Albert Einstein?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 


pergunta: Quem foi o pai de Napoleão Bonaparte?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonapar

In [49]:
llm.invoke(prompt.format(input="quem fez mais gols messi ou cr7?"))

NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

### Few shot prompt em chat


In [51]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(api_key=os.getenv("openaiKey"))

In [52]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
    ("human","{pergunta}"),
    ("ai","{resposta}")
])

In [53]:
few_shot_template = FewShotChatMessagePromptTemplate(examples=exemplos,
                                                     example_prompt=example_prompt)

prompt_final = ChatPromptTemplate.from_messages([
    few_shot_template,
    ("human","{input}")
])

In [54]:
prompt = prompt_final.format_messages(input="qual jogaador de futebol tem mais bola de ouro?")

In [55]:
chat.invoke(prompt)

AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quem tem mais Bolas de Ouro? \nResposta intermediária: Lionel Messi e Cristiano Ronaldo são os jogadores de futebol com mais Bolas de Ouro, ambos venceram o prêmio várias vezes. \nEntão a resposta final é: Lionel Messi e Cristiano Ronaldo', response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 522, 'total_tokens': 607, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7a2ebbc1-517f-404e-a824-e133bcba5901-0')